In [1]:
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset

In [2]:
df = pd.read_csv('~/data/FEATURES.csv')

In [3]:
df.head()

,patientID,hadmID,chartTime,minSystolicBP,maxSystolicBP,minPulsePressure,maxPulsePressure,minHeartRate,maxHeartRate,minRespRate,maxRespRate,minTempC,maxTempC,minSpO2,minGCS,hourDelta,isSeptic
0,5215,136637,4455392400000,112.0,112.0,17.0,17.0,95.0,95.0,16.0,16.0,37.0,37.0,96.5,15.0,1,0
1,5215,136637,4455396000000,110.0,112.0,17.0,30.0,92.0,95.0,16.0,19.0,37.0,37.0,97.0,12.0,2,0
2,5215,136637,4455399600000,110.0,112.0,17.0,65.0,77.0,95.0,16.0,19.0,37.0,37.0,95.0,12.0,3,0
3,5215,136637,4455403200000,110.0,112.0,17.0,70.0,73.0,95.0,15.0,19.0,37.0,37.0,95.0,12.0,4,0
4,5215,136637,4455406800000,110.0,113.0,17.0,70.0,73.0,95.0,15.0,19.0,37.0,37.0,95.0,12.0,5,0


In [4]:
groups = df.groupby(['patientID', 'hadmID']).groups

In [5]:
indexesForEachPatient = list(groups.values())

In [6]:
featureColumns = ['minSystolicBP', 'maxSystolicBP', 'minPulsePressure',
                  'maxPulsePressure', 'minHeartRate', 'maxHeartRate',
                 'minRespRate', 'maxRespRate', 'minTempC', 'maxTempC',
                  'minSpO2', 'minGCS', 'hourDelta']
examples = []
labels = []

# get the features extracted and the label
for i in indexesForEachPatient:
    sortedDf = df.loc[i].sort_values(by='hourDelta')
    rowsForPatient = sortedDf[featureColumns].values
    examples.append(rowsForPatient)
    labels.append(sortedDf['isSeptic'].iloc[0])

In [22]:
maxLength = max([len(l) for l in examples])
numFeatures = len(examples[0][0])

In [30]:
paddedValues = []
for v in examples:
    howManyNewPoints = maxLength - len(v)
    toAdd = np.zeros((howManyNewPoints, numFeatures))
    paddedValues.append(np.append(v, toAdd, axis=0))

In [31]:
paddedValues

[array([[ 62.,  62.,  38., ...,  98.,  15.,   0.],
        [ 62., 217.,  28., ...,  74.,  15.,   1.],
        [ 62., 217.,  28., ...,  74.,  15.,   2.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]]),
 array([[120., 120.,  50., ...,  98.,  15.,   0.],
        [116., 116.,  53., ...,  98.,  15.,   1.],
        [116., 116.,  53., ...,  98.,  15.,   2.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]]),
 array([[125., 125.,  69., ...,  99.,  15.,   0.],
        [125., 141.,  69., ...,  99.,  15.,   1.],
        [125., 141.,  69., ...,  99.,  15.,   2.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]]),
 array([[120., 203.,  56., ...,  97.,

In [32]:
torch.from_numpy(np.array(paddedValues))

tensor([[[ 62.,  62.,  38.,  ...,  98.,  15.,   0.],
         [ 62., 217.,  28.,  ...,  74.,  15.,   1.],
         [ 62., 217.,  28.,  ...,  74.,  15.,   2.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[120., 120.,  50.,  ...,  98.,  15.,   0.],
         [116., 116.,  53.,  ...,  98.,  15.,   1.],
         [116., 116.,  53.,  ...,  98.,  15.,   2.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[125., 125.,  69.,  ...,  99.,  15.,   0.],
         [125., 141.,  69.,  ...,  99.,  15.,   1.],
         [125., 141.,  69.,  ...,  99.,  15.,   2.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],